In [19]:
import xarray as xr
import math
import numpy as np
from datetime import datetime as dt
import datetime
from matplotlib import pyplot as plt
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

np.set_printoptions(threshold=5000)

%matplotlib inline

In [9]:
flFile = '/Users/danstechman/GoogleDrive/VSE-Data/flightLevel/20170422H1_AC.nc'
flData = xr.open_dataset(flFile,decode_times=False)

## Vars from HH/MM/SS

In [13]:
flDate = flData.FlightDate
HH = flData.get('HH').to_masked_array()
MM = flData.get('MM').to_masked_array()
SS = flData.get('SS').to_masked_array()

if flDate[2] == '/':
    crntDay = dt.strptime(flDate,'%m/%d/%Y')
else:
    crntDay = dt.strptime(flDate,'%Y-%m-%d')
    
if np.min(np.diff(HH)) == -23.0:
    crsMidnt = True
    midntIx = np.where(np.diff(HH) == -23.0)[0][0] + 1
    nxtDay = crntDay + datetime.timedelta(days=1)
else:
    crsMidnt = False

hms_dt = np.empty(np.shape(HH),dtype=object)

if crsMidnt:
    for idt in np.arange(np.size(HH[:midntIx])):
        hms_dt[idt] = dt.strptime('{:%Y%m%d}{:02.0f}{:02.0f}{:02.0f}'.format(crntDay,HH[idt],MM[idt],SS[idt]),'%Y%m%d%H%M%S')
    for idt in range(np.size(HH[:midntIx]),np.size(HH)):
        hms_dt[idt] = dt.strptime('{:%Y%m%d}{:02.0f}{:02.0f}{:02.0f}'.format(nxtDay,HH[idt],MM[idt],SS[idt]),'%Y%m%d%H%M%S')
else:
    for idt in np.arange(np.size(HH)):
        hms_dt[idt] = dt.strptime('{:%Y%m%d}{:02.0f}{:02.0f}{:02.0f}'.format(crntDay,HH[idt],MM[idt],SS[idt]),'%Y%m%d%H%M%S')

hms_dt_Mskd = np.ma.MaskedArray(hms_dt,mask=np.ma.getmaskarray(HH),fill_value='NaT')
hms_HH = np.asarray([int(dt.strftime(d,'%H')) for d in hms_dt])

In [21]:
hms_dt_Mskd.data[:4500]

array([datetime.datetime(2017, 4, 22, 0, 0),
       datetime.datetime(2017, 4, 22, 0, 0),
       datetime.datetime(2017, 4, 22, 0, 0),
       datetime.datetime(2017, 4, 22, 0, 0),
       datetime.datetime(2017, 4, 22, 0, 0),
       datetime.datetime(2017, 4, 22, 0, 0),
       datetime.datetime(2017, 4, 22, 15, 59, 31),
       datetime.datetime(2017, 4, 22, 15, 59, 32),
       datetime.datetime(2017, 4, 22, 15, 59, 33),
       datetime.datetime(2017, 4, 22, 15, 59, 34),
       datetime.datetime(2017, 4, 22, 15, 59, 35),
       datetime.datetime(2017, 4, 22, 15, 59, 36),
       datetime.datetime(2017, 4, 22, 15, 59, 37),
       datetime.datetime(2017, 4, 22, 15, 59, 38),
       datetime.datetime(2017, 4, 22, 15, 59, 39),
       datetime.datetime(2017, 4, 22, 15, 59, 40),
       datetime.datetime(2017, 4, 22, 15, 59, 41),
       datetime.datetime(2017, 4, 22, 15, 59, 42),
       datetime.datetime(2017, 4, 22, 15, 59, 43),
       datetime.datetime(2017, 4, 22, 15, 59, 44),
       datetime.

In [35]:
time_dt64 = flData.get('Time').to_masked_array()

In [36]:
time_dt64 = time_dt64.filled(fill_value=0)

In [37]:
timeFL = [dt.strptime('{:%Y%m%d}155931'.format(crntDay),'%Y%m%d%H%M%S') + datetime.timedelta(seconds=int(t)) for t in time_dt64]

In [38]:
timeFL

[datetime.datetime(2017, 4, 22, 15, 59, 31),
 datetime.datetime(2017, 4, 22, 15, 59, 31),
 datetime.datetime(2017, 4, 22, 15, 59, 31),
 datetime.datetime(2017, 4, 22, 15, 59, 31),
 datetime.datetime(2017, 4, 22, 15, 59, 31),
 datetime.datetime(2017, 4, 22, 15, 59, 31),
 datetime.datetime(2017, 4, 22, 15, 59, 31),
 datetime.datetime(2017, 4, 22, 15, 59, 31),
 datetime.datetime(2017, 4, 22, 15, 59, 39),
 datetime.datetime(2017, 4, 22, 15, 59, 40),
 datetime.datetime(2017, 4, 22, 15, 59, 41),
 datetime.datetime(2017, 4, 22, 15, 59, 42),
 datetime.datetime(2017, 4, 22, 15, 59, 43),
 datetime.datetime(2017, 4, 22, 15, 59, 44),
 datetime.datetime(2017, 4, 22, 15, 59, 45),
 datetime.datetime(2017, 4, 22, 15, 59, 46),
 datetime.datetime(2017, 4, 22, 15, 59, 47),
 datetime.datetime(2017, 4, 22, 15, 59, 48),
 datetime.datetime(2017, 4, 22, 15, 59, 49),
 datetime.datetime(2017, 4, 22, 15, 59, 50),
 datetime.datetime(2017, 4, 22, 15, 59, 51),
 datetime.datetime(2017, 4, 22, 15, 59, 52),
 datetime.

## Vars from Time

In [4]:
time_dt64 = flData.get('Time').to_masked_array()

time_tStmp = (time_dt64 - np.datetime64('1970-01-01T00:00:00')) / np.timedelta64(1, 's')
time_dt = np.asarray([dt.utcfromtimestamp(t) for t in time_tStmp])
time_hh = np.asarray([int(dt.strftime(d,'%H')) for d in time_dt])

time_dt_Mskd = np.ma.MaskedArray(time_dt,mask=np.ma.getmaskarray(HH),fill_value='NaT')

In [5]:
T_offset = hms_dt_Mskd - time_dt_Mskd

if np.array_equal(time_dt_Mskd,hms_dt_Mskd):
    print('Flight-level Time and aggregated HH/MM/SS agree')
else:
    sec_offset = np.asarray([t.total_seconds() for t in T_offset.filled(datetime.timedelta(seconds=-9.999))])
    sec_offset[sec_offset == -9.999] = np.nan
    
    if np.nanmin(sec_offset) == np.nanmax(sec_offset):
        if np.abs(np.nanmax(sec_offset)) >= 60:
            ofst = np.nanmax(sec_offset)/60/60
            print('Aggregated HH/MM/SS has a {} hour offset relative to Time variable'.format(ofst))
        else:
            print('Aggregated HH/MM/SS has a {} second offset relative to Time variable'.format(np.nanmax(sec_offset)))
    else:
        print('Aggregated HH/MM/SS has a non-constant offset relative to Time variable')
        print('\t(HMS - Time) minimum = {} seconds'.format(np.nanmin(sec_offset)))
        print('\t(HMS - Time) maximum = {} seconds'.format(np.nanmax(sec_offset)))

Aggregated HH/MM/SS has a -6.0 second offset relative to Time variable


In [6]:
# plt.plot(dtArr,time_hh)
# plt.plot(dtArr,HH)
# plt.plot(dtArr[np.where(T_offset != 0)],time_hh[np.where(T_offset != 0)],'b.')

# allTrace = go.Scatter(x = flDT, y = flRoll, connectgaps = False, hoverinfo='x+y')
# markTrace = go.Scatter(x = flDT[rollExcdIx], y = flRoll[rollExcdIx], mode = 'markers',hoverinfo = 'skip')
# pltData = [allTrace, markTrace]

# iplot(pltData)